# Автоматическая обработка языка
## НИУ ВШЭ, 2020-2021 учебный год

### Домашнее задание №3

Задание выполнил(а): Дарья Матяш

Ссылка на условия и требования: https://github.com/named-entity/hse-nlp/blob/master/DZ3_mod.ipynb

В этой домашней работе вам предстоит составить csv или pandas таблицу, в которой будет хранится информация о широком или узком топике текста.
Мы будем работать с тем же датасетом, что и на последнем семинаре по gensim.
Для начала определим широкий топик текста с помощью gensim. Так, На семинаре мы познакомились с "обычным" встроенным дженсимом, но у него есть улучшенная версия - mallet, для запуска которой надо скачать zip file, распаковать его, дать обертке для маллета path к файлу.... и просто запустить.

За **"прогон" дженсима** в его базовой версии вы получите - один балл, за прогон **с маллетом** - **2** (1 основной и один бонусный).
**Создайте функцию или серию функций**, через которую будет удобно **подобрать оптимальное число групп** - **1 балл** за нахождение оптимального числа групп, **1 балл** - если этобудет не руками, а через **функцию** **2 балла**


Как вы знаете, gensim считает, что каждый текст содержит несколько топиков, но на следующем этапе вам будет надо создать функцию, которая будет для каждого текста определять один широкий топик, самый главный.
Мы предлагаем сделать это так: создайте счётчик, и каждый раз, когда в тексте будет встречаться одно из слов, соответсвующих данной теме, добавляйте к счетчику его вес. **2 балла**


После того, как у вас получится какое-то количество групп (наборов текстов с общим топиком). Внутри каждой из этих групп посчитайте **тф_идф для каждого текста** (Т. е. возьмите все тексты с одинаковой темой за ваш корпус и посчитайте для каждого из этих текстов тф_идф). **3 балла**
Каждому тексту определите **слова с пятью самыми высокими тф_идф** и запишите их в таблицу
вывод - excel, csv или pandas таблица с текстом, его широким топиком по дженсиму и 5 тф_идф словами для этого текста - **1 балл**
Логичность и красота кода - **1 балл**
ещё один бонус балл: описать как работает coherence score (не успели обсудить на паре) на русском - словами
дэдлайн 19 окт 23:59

In [272]:
import pandas as pd
import numpy as np
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import nltk
from nltk.corpus import stopwords
import spacy

In [48]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [49]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
df.head()

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [50]:
print('Изначальное число топиков в датасете:', len(df.target_names.unique()))

Изначальное число топиков в датасете: 20


In [51]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))


In [53]:
content_tokens = list(sent_to_words(df['content'].to_list()))

In [54]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [55]:
# Remove Stop Words
data_words_nostops = remove_stopwords(content_tokens)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['thing', 'car', 'nntp', 'posting', 'host', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'know', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [110]:
df['data_lemmatized'] = data_lemmatized

In [56]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

Мы знаем, что изначальное число топиков в датасете - 20, поищем оптимальное автоматические разбиение на топики от 1 до 30 возможных топиков

In [57]:
mallet_path = 'path/to/mallet-2.0.8/bin/mallet'

In [70]:
coherence_scrores = []
for i in range(1,30):
    mallet_path = '/Users/pikachu/Downloads/mallet-2.0.8/bin/mallet'
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=i, id2word=id2word)
    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_scrores.append(coherence_model_ldamallet.get_coherence())
    

In [289]:
print('оптимальное число топиков: ', coherence_scrores.index(max(coherence_scrores)))

оптимальное число топиков:  22


In [290]:
num_topics = coherence_scrores.index(max(coherence_scrores))

In [292]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)

Посмотрим, какие топики с какими словами выделены (по умолчанию на каждый топик выводится по 10 слов - мне кажется, это отлично

In [293]:
topics_dict = {}
topics_weights = {}
for i in range(len(ldamallet.show_topics(formatted=False, num_topics=num_topics))):
    topics_dict[i] = list([tup[0] for tup in ldamallet.show_topics(formatted=False, num_topics=num_topics)[i][1]])
    topics_weights[i] = list([tup[1] for tup in ldamallet.show_topics(formatted=False, num_topics=num_topics)[i][1]])

Предскажем для каждого текста его топик:

In [294]:
def predict_topic(x):
    ans_dict = {i: 0 for i in range(22)}
    for word in x:
        for i in range(num_topics):
            if word in topics_dict[i]:
                ans_dict[i] += topics_weights[i][topics_dict[i].index(word)]
    return max(ans_dict, key=ans_dict.get)

In [295]:
df['predicted_topic'] = df['data_lemmatized'].apply(predict_topic)

In [296]:
df

,content,target,target_names,data_lemmatized,predicted_topic
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos,"[thing, car, nntp, posting, host, line, wonder...",10
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware,"[final, summary, final, call, si, clock, repor...",10
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware,"[question, computer, network, distribution, li...",10
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics,"[division, line, distribution, world, nntp, po...",12
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space,"[article, owcb, world, std, com, tombaker, wor...",12
...,...,...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13,sci.med,"[factory, scan, distribution, factory, line, c...",10
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,4,comp.sys.mac.hardware,"[problem, screen, blank, sometimes, minor, phy...",13
11311,From: westes@netcom.com (Will Estes)\nSubject:...,3,comp.sys.ibm.pc.hardware,"[este, mount, case, line, instal, try, mount, ...",10
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1,comp.graphics,"[line, post, host, write, boy, embarasse, triv...",10


Сгруппируем все тексты по их предсказанным топикам:

In [281]:
d = df.groupby(['predicted_topic'])['data_lemmatized'].apply(lambda x: list(x.value_counts().index)).to_dict()

Сделаем tf-idf представление слов внутри каждой подгруппы топиков и выведем топ-5 

In [283]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [302]:
d_ind = list(d.keys())

In [ ]:
top5_dict = {}
for i in d_ind:
    cur_corpus = [' '.join(sent) for sent in d[i]]
    X = vectorizer.fit_transform(cur_corpus)
    sents = []
    for j in range(len(X.toarray())):
        ans = []
        for nums in X.toarray()[j].argsort()[-5:]:
            ans.append(vectorizer.get_feature_names()[nums])
        sents.append(ans)
    top5_dict[i] = sents

Создадим датафрейм "текст (выдаем списком, можно и не списком, но мне так проще) - определенный топик - топ-5 слов текста с самым высоким tf-idf в его группе топиков"

In [ ]:
ans_text = []
ans_topics =[]
ans_top5 = []
for i in range(len(d)):
    for j in range(len(d[i])):
        ans_text.append(d[i][j])
        ans_topics.append(i)
        ans_top5.append(top5_dict[i][j])

In [ ]:
df_ans = pd.DataFrame.from_dict({'ans_text': ans_text,
         'ans_topics': ans_topics,
         'ans_top5': ans_top5})

In [ ]:
df_ans

In [ ]:
df_ans.to_csv('NLP-HW-03.csv', encoding='utf-8')

#### Описание работы **Coherence Score**:

**Определение:** Топик называется когерентным (согласованным), если термины, наиболее частые в данном топике, неслучайно часто совместно встречаются рядом в документах коллекции. 
Когерентность может оцениваться по сторонней коллекции, либо по той же коллекции, по которой строится модель(в нашем случае такой вариант).

Для оценивания когерентности используется поточечная взаимная информация (pointwise mutual information, PMI):
на примере двух тем А и В покажу, как это рассчитвается - 
логарифм (с любым основанием, на самом деле)((вероятность термина A встретиться вместе/рядом с термином B) / произведение априорных вероятностей появления термина А и B соответственно в обучающей выборке (отношение количества вхождений к общему количеству слов в корпусе))

log(P(A near B)/P(A) * P(B))

Примечание: (A near B) — вероятность термина A встретиться вместе/рядом с термином B; «рядом» можно конфигурировать вручную, по умолчанию расстояние равно 10 терминам влево и вправо; основание логарифма не играет роли, для простоты примем его равным 2. 

Позитивный знак логарифма будет означать положительны окрас A по сравнению с B, негативный — отрицательный. 


Например, чтобы определить к какому классу относятся высказывания «хорошая погода», «быстро ехать», достаточно проверить в обучающей выборке как часто «хорошая погода» и «быстро ехать» встречается рядом с заведомо (установленными человеком в зависимости от модели данных и тестовой выборки) хорошими и плохими словами и установить разницу. 



Литература:
http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf


на примере с водой очень понравилось:

Segmentation: A lot of dispatch product divided into different sub-lot sizes, such that each sub-lot product are different.
Probability Estimation: Quantitative Measurement of sub lot quality.
Confirmation Measure: Determine quality as per some predefined standard (say % conformance) and assign some number to qualify. For example, 75% of products are good quality as per XXX standard.